In [1]:
# imports

import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"

messages = [{"role": "user", "content": message}]

messages


[{'role': 'user',
  'content': 'Hello, GPT! This is my first ever message to you! Hi!'}]

In [4]:
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
response.choices[0].message.content

"Hello! Welcome! I'm glad to hear from you. How can I assist you today?"

In [5]:
# Let's try out this utility

ed = fetch_website_contents("https://edwarddonner.com")
print(ed)

Home - Edward Donner

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Conne

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [6]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [7]:
# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [8]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

response = openai.chat.completions.create(model="gpt-4.1-nano", messages=messages)
response.choices[0].message.content

'2 + 2 equals 4.'

## And now let's build useful messages for GPT-4.1-mini, using a function

In [9]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [10]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': '\nYou are a snarky assistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\n\nHome - Edward Donner\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n.

## Time to bring it together - the API for OpenAI is very simple!

In [11]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(
        model = "gpt-4.1-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [12]:
summarize("https://edwarddonner.com")

'# Edward Donner’s LLM Playground & AI Bragging Rights\n\nWelcome to Ed’s corner of the web where he flexes his coder muscles and obsessively tinkers with Large Language Models. Besides awkwardly DJing and making cringe-worthy electronic beats, he’s the CTO of Nebula.io, which is basically AI matchmaking for recruiters—not your typical dating app drama, just helping companies find employees who don’t hate their jobs immediately.\n\nHe’s got some fancy patents and a track record of selling his AI startup untapt, proving he’s less amateur than he claims. Newsflash: Ed is hosting AI events, talking big about AI leadership curriculums, and dropping executive briefings for 2025—because apparently that’s what real AI wizards do.\n\nAlso, there’s a cheeky LLM smackdown called “Outsmart,” where bots duke it out in diplomacy and cunning. If you dig that sort of digital gladiator action or just want to see which AI is the sassiest, this is your stop.\n\nP.S. He’s not-so-subtly fishing for connec

In [13]:
# A function to display this nicely in the output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [14]:
display_summary("https://edwarddonner.com")

# Edward Donner’s Corner of the Internet

Welcome to the lair of Ed, a code junkie and LLM tinkerer who’s more into AI than your average tech geek—and yes, he DJs, but don’t expect a mixtape anytime soon. He’s the CTO/co-founder of Nebula.io, where AI isn’t just buzzwords but actually helps recruiters find talent without the usual headache. Fun fact: Ed founded an AI startup called untapt that got scooped up in 2021, proving he’s got the Midas touch.

Oh, and if you’re here for some intellectual entertainment, check out his “Outsmart” arena where language models duke it out in diplomatic shenanigans. Plus, there’s Connect Four for a little old-school fun.

News bite highlights:
- November 2025: Feels the “Unique Energy of an AI Live Event” — sounds electrifying!
- September 2025: Dropped wisdom on scaling Gen AI and Agentic AI on AWS.
- May 2025: Shares a “Be an AI Engineer and Leader” curriculum and spills the beans at the 2025 AI Executive Briefing.

Mostly, it’s a neat blend of AI geekery, professional hustle, and gentle DJ self-deprecation. Sign up for his newsletter if you want to keep pretending you understand AI as much as Ed does.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [15]:
display_summary("https://cnn.com")

# CNN: The Place Where Everything Happens... and Then Some

Welcome to CNN, your one-stop shop for *literally* every category you can think of — from "US Politics" to "Redistricting Tracker" (thrilling stuff), plus hotspots like "Ukraine-Russia War" and "Israel-Hamas War" for your daily dose of global tension.

Got a thing for sports? They've got football, tennis, golf, and even climbing (because why not?). Into lifestyle? There’s fashion, luxury, mindfulness, and "Life, But Better" — basically everything designed to make you feel like you’ve got your act together.

Oh, and they apparently care *a lot* about your ad experience — because who doesn’t want to give feedback when that video ad freezes for the 10th time?

No actual breaking news or new announcements here though, just the usual buffet of categories and a polite request to rate ads. Basically, CNN is like that friend who’s always watching the news, knows a bit about everything, and occasionally asks if you’re enjoying the commercials.

In [16]:
display_summary("https://anthropic.com")

# Anthropic: Safe AI or Just Playing It Safe?

Welcome to Anthropic, the AI playground for safety nerds who want to make sure their fancy language models don't accidentally take over the world. They’re all about researching AI that’s both powerful and polite—because apparently, our robot overlords need a code of ethics.

**What’s cooking?**  
- Meet *Claude Sonnet 4.5* — the "best model in the world" for agents, coding, and computer use. So basically, your new AI sidekick who might code better than you (ouch).  
- Also, *Claude Haiku 4.5* is in the mix, managing context like a pro on their developer platform — goodbye confusion, hello efficiency.  

**Big Ideas:**  
Anthropic stresses the need for *bold progress* along with *intentional pauses* to contemplate AI’s existential impact, which is fancy talk for “We’re innovating, but also trying not to blow up the planet.”

**In a nutshell:**  
A serious AI company obsessed with making sure AI is safe, transparent, and used for good, wrapped in a corporate shroud of public benefit. Oh, and they want you to try Claude and maybe download their app if you’re feeling adventurous.

So if your AI needs a responsible babysitter, Anthropic’s your go-to.

In [17]:
# Step 1: Create your prompts

system_prompt = "something here"
user_prompt = """
    Lots of text
    Can be pasted here
"""

# Step 2: Make the messages list

messages = [] # fill this in

# Step 3: Call OpenAI
# response =

# Step 4: print the result
# print(